In [4]:
import openai, json

client = openai.OpenAI()

messages = []


In [5]:
def get_weather(city):
    return "33degrees celcius."


FUNCTION_MAP = {"get_weather": get_weather}

In [6]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather for.",
                    }
                },
                "required": ["city"]
            }
        }
    }
]

In [7]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"🔷 -- Calling function: {function_name} with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result or "",
            })

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"--- 👁️ AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [8]:
while True:
    userMessage = input("Send a message to the LLM... ")

    if userMessage == "quit" or userMessage == "q":
        break
    else:
        messages.append({"role": "user", "content": userMessage})
        print(f"--- 🧑‍🦲 User: {userMessage}")
        call_ai()

--- 🧑‍🦲 User: 오늘 날씨 어때?
--- 👁️ AI: 어떤 도시의 날씨를 알고 싶으신가요? 특정 도시를 말씀해 주시면 그곳의 날씨 정보를 제공해 드리겠습니다.
--- 🧑‍🦲 User: 상하이
🔷 -- Calling function: get_weather with arguments: {"city":"상하이"}
Ran get_weather with args {'city': '상하이'} for a result of 33degrees celcius.
--- 👁️ AI: 상하이의 현재 기온은 33도 섭씨입니다. 다른 날씨 정보가 필요하시면 말씀해 주세요!
